numerical_cols

In [1]:
from pipeline_perso import CustomImputer
import numpy as np
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from pipeline_perso import CategoricalTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR, SVC, LinearSVC
from pipeline_perso import models_training_using_pipeline

In [2]:
X_full=pd.read_csv('kidney_disease.csv')
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['classification'], inplace=True)
y = X_full.classification
X_full.drop(['classification'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()


In [3]:


num_imputer=CustomImputer(missing_values=np.nan, strategy='mean')
cat_imputer=Pipeline(steps=[
    ('imputer', CustomImputer(missing_values=np.nan, strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, numerical_cols),
        ('cat', cat_imputer, categorical_cols),
       
    ])


In [4]:
#example of how we can use the custom imputer on numerical data
num_imputer=CustomImputer(missing_values=np.nan, strategy='mean')
num_imputer.transform(X_train[numerical_cols])

array([[336.        ,  25.        ,  60.        , ..., 137.58669355,
          4.68987854,  15.2       ],
       [ 64.        ,  55.        ,  80.        , ..., 137.58669355,
          4.68987854,   9.8       ],
       [ 55.        ,  35.        ,  80.        , ..., 137.58669355,
          4.68987854,   9.5       ],
       ...,
       [117.        ,  50.99041534,  70.        , ..., 139.        ,
          3.7       ,  12.5       ],
       [ 47.        ,  11.        ,  80.        , ..., 137.58669355,
          4.68987854,  15.        ],
       [172.        ,  62.        ,  80.        , ..., 130.        ,
          2.5       ,  10.6       ]])

In [5]:
#models to evaluate
models = []
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
#testing models_training_using_pipeline fonction
models_training_using_pipeline(preprocessor,models,X_train,y_train,X_valid,y_valid)

accuracy score for model : LDA is 0.975
accuracy score for model : KNN is 0.975
accuracy score for model : CART is 0.9875
accuracy score for model : NB is 0.975
accuracy score for model : SVM is 0.9625
